In [2]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("/nfs/turbo/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl")
from travel import init_travel
init_travel()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load preds

In [3]:
import json
from pprint import pprint

RESULTS_PATH_LIKELIHOOD = "/home/sstorks/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl/saved_results_222/vqa_mistake_detection/ego4d_single_debug250/llava-1.5-7b-hf/IterativeVQA_q10_ego4d_single_debug250_llava-1.5-7b-hf_beam8-4_likelihood_nohistory_20240815204213"
preds_l = json.load(open(os.path.join(RESULTS_PATH_LIKELIHOOD, "outputs_val.json"), "r"))
metrics_l = json.load(open(os.path.join(RESULTS_PATH_LIKELIHOOD, "metrics_coherence_nli_val.json"), "r"))
combined_l = [v | {"example_id": k, 
                   "relevance_marginal": metrics_l['metrics_by_turn']['relevance_marginal_by_turn'][i], 
                   "informativeness_marginal": metrics_l['metrics_by_turn']['informativeness_marginal_by_turn'][i],
                   "informativeness_marginal_x_relevance_marginal": metrics_l['metrics_by_turn']['informativeness_marginal_x_relevance_marginal_by_turn'][i],
                   "informativeness_marginal_x_relevance_marginal_ref": metrics_l['metrics_by_turn']['informativeness_marginal_x_relevance_marginal_ref_by_turn'][i]
                  } for i, (k, v) in enumerate(preds_l.items())]

combined_c = []
RESULTS_PATH_COHERENCE = "/home/sstorks/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl/saved_results_222/vqa_mistake_detection/ego4d_single_debug250/llava-1.5-7b-hf/IterativeVQA_q10_ego4d_single_debug250_llava-1.5-7b-hf_beam8-4_coherence_icl20_nohistory_20240815204213"
preds_c = json.load(open(os.path.join(RESULTS_PATH_COHERENCE, "outputs_val.json"), "r"))
metrics_c = json.load(open(os.path.join(RESULTS_PATH_COHERENCE, "metrics_coherence_nli_val.json"), "r"))
combined_c = [v | {"example_id": k, 
                   "relevance_marginal": metrics_c['metrics_by_turn']['relevance_marginal_by_turn'][i], 
                   "informativeness_marginal": metrics_c['metrics_by_turn']['informativeness_marginal_by_turn'][i],
                   "informativeness_marginal_x_relevance_marginal": metrics_c['metrics_by_turn']['informativeness_marginal_x_relevance_marginal_by_turn'][i],
                   "informativeness_marginal_x_relevance_marginal_ref": metrics_c['metrics_by_turn']['informativeness_marginal_x_relevance_marginal_ref_by_turn'][i],
                  } for i, (k, v) in enumerate(preds_c.items())]

all_data = combined_l + combined_c

pprint(all_data[0])

{'answer_probs': [[0.010489, 0.989511],
                  [0.7773, 0.2227],
                  [0.007233, 0.992767],
                  [0.651355, 0.348645],
                  [0.307358, 0.692642],
                  [0.33112, 0.66888],
                  [0.573678, 0.426322],
                  [0.73412, 0.26588],
                  [0.538983, 0.461017],
                  [0.72487, 0.27513]],
 'answers': ['Yes',
             'No',
             'Yes',
             'No',
             'Yes',
             'Yes',
             'Unsure',
             'No',
             'Unsure',
             'No'],
 'candidate_questions': [['Is the person wearing a lab coat?',
                          'Is the person wearing gloves?',
                          'Is the person wearing a white lab coat?'],
                         ['Is the person wearing safety goggles?',
                          'Is the person wearing gloves?'],
                         ['Is the person wearing gloves?',
                          'I

# Select turns and candidates to score

Relevance study:

In [4]:
import datetime
import os
import random

from travel import set_random_seed
from travel.constants import RANDOM_SEED, RESULTS_DIR

N_SAMPLES = 50
SAMPLES_PER_ANNOTATOR = 10

timestamp = datetime.datetime.now()
output_dir = os.path.join("annotation_files", f"relevance_{timestamp.strftime('%Y%m%d%H%M%S')}")
os.makedirs(output_dir)

random_seed = 555
set_random_seed(random_seed) # Set seed to 555 for relevance samples

example_idxs = random.sample(list(range(len(all_data))), N_SAMPLES)

data_dicts = []
for example_idx in example_idxs:   
    output = all_data[example_idx]
        
    example_id = output['example_id']

    n_turns = output['final_turn'] + 1
    selected_turn = random.choice(list(range(n_turns)))

    selected_question = output['questions'][selected_turn]
    selected_answer = output['answers'][selected_turn]
    selected_answer = "Yes" if selected_answer == 1 else "No"

    previous_questions = output['questions'][:selected_turn]
    previous_answers = ["Yes" if a == 1 else "No" for a in output['answers'][:selected_turn]]

    actual_relevance_marginal = output['relevance_marginal'][selected_turn]
    actual_informativeness_marginal = output['informativeness_marginal'][selected_turn]
    actual_informativeness_marginal_x_relevance_marginal = output['informativeness_marginal_x_relevance_marginal'][selected_turn]
    actual_informativeness_marginal_x_relevance_marginal_ref = output['informativeness_marginal_x_relevance_marginal_ref'][selected_turn]
                                  
    data_dicts.append(
        {
            "example_id": example_id,
            "procedure": output['procedure'],
            "question": selected_question,
            "previous_questions_answers": [(q, a) for q, a in zip(previous_questions, previous_answers)],
            "nli_metric_scores": {
                "relevance_marginal": actual_relevance_marginal,
                "informativeness_marginal": actual_informativeness_marginal,
                "informativeness_marginal_x_relevance_marginal": actual_informativeness_marginal_x_relevance_marginal,
                "informativeness_marginal_x_relevance_marginal_ref": actual_informativeness_marginal_x_relevance_marginal_ref,
            }
        }
    )

json.dump(data_dicts, open(os.path.join(output_dir, f"relevance_sample{random_seed}.json"), "w"), indent=4)

In [4]:
# Divide sampled data into a txt file for each annotator
for annotator_idx in range(N_SAMPLES // SAMPLES_PER_ANNOTATOR):
    lines = []

    samples = data_dicts[annotator_idx * SAMPLES_PER_ANNOTATOR:(annotator_idx + 1) * SAMPLES_PER_ANNOTATOR]
    assert len(samples) == SAMPLES_PER_ANNOTATOR
    
    lines.append("Imagine you just had eye surgery, and are unable to see. You're performing a task you're familiar with, but need help to determine whether you successfully completed it.")
    lines.append("You video call a friend (who is unfamiliar with the task) and show them what you're working on. You then ask them some yes/no questions to figure out whether you successfully completed the task.")
    lines.append("")
    lines.append("For each annotation task, you will be given the following information:")
    lines.append(" - A sentence describing the procedure you're trying to perform.")
    lines.append(" - An optional list of previous questions you already asked, and their answers.")
    lines.append(" - A potential question you could ask your friend next.")
    lines.append("")
    lines.append("You must rate how relevant the potential next question is. By relevant, we mean: given the previous questions and answers, how helpful could an answer to this question be in determining whether the procedure has been completed?")
    lines.append("You can also choose to mark 'Instructions Unclear', which means that the instructional text itself is not clear, so you're not sure how to determine whether it's successful. This should only be used in rare cases.")
    lines.append("")
    lines.append("Some tips:")
    lines.append(" - Only judge the relevance of the potential next question, not the previous questions (which may or may not be relevant).")
    lines.append(" - A question may seem relevant to the task at hand, but you should consider it irrelevant if it doesn't provide essential information to judge the success of the task.")
    lines.append(" - If a seemingly relevant question is redundant with previous questions, you may consider it less relevant.")
    lines.append(" - Assume that the answer to the question won't contradict the information you have from previous questions and answers. If previous questions and answers already contradict each other, consider whether this question could sway you one way or another.")
    lines.append(' - The instructional text and questions may refer to "someone" or a "person"; always assume this is referring to yourself (the person performing the task).')
    lines.append(' - The questions may refer to a "photo" or "image"; always assume this is referring to the video feed your friend would see through the video call.')
    lines.append("\n")
                 
    # How relevant is this question, i.e., how useful would an answer to this question be in determining whether the procedure has been completed?
    for sample_idx, sample in enumerate(samples):
        lines.append(f"Annotation {sample_idx + 1}")
        lines.append("===============================")
        lines.append(f"Instruction: {sample['procedure']}")
        
        lines.append("Previous questions and answers:")
        if len(sample['previous_questions_answers']) == 0:
            lines.append("None")
        else:
            for q_idx, (q, a) in enumerate(sample['previous_questions_answers']):
                 lines.append(f"Q{q_idx+1}. {q}     (Answer: {a})")
        lines.append("")
        lines.append(f"Potential next question: {sample['question']}")
        lines.append("Your rating (place an 'x' next to your choice):")
        lines.append("1 (very irrelevant): ")
        lines.append("2 (slightly irrelevant): ")
        lines.append("3 (neutral): ")
        lines.append("4 (slightly relevant): ")
        lines.append("5 (very relevant): ")
        lines.append("Instructions Unclear: ")
        lines.append("\n")
                 
    with open(os.path.join(output_dir, f"relevance_sample{random_seed}_annotator{annotator_idx+1}.txt"), "w") as f:
        f.write("\n".join(lines))

Informativeness study:

In [5]:
import datetime
import os
import random

from travel import set_random_seed
from travel.constants import RANDOM_SEED, RESULTS_DIR

N_SAMPLES = 50
SAMPLES_PER_ANNOTATOR = 10

timestamp = datetime.datetime.now()
output_dir = os.path.join("annotation_files", f"informativeness_{timestamp.strftime('%Y%m%d%H%M%S')}")
os.makedirs(output_dir)

random_seed = 888
set_random_seed(random_seed) # Set seed to 888 for informativeness samples

example_idxs = random.sample(list(range(len(all_data))), N_SAMPLES)

data_dicts = []
for example_idx in example_idxs:
    output = all_data[example_idx]
    example_id = output['example_id']

    n_turns = output['final_turn'] + 1
    selected_turn = random.choice(list(range(n_turns)))

    selected_question = output['questions'][selected_turn]
    selected_answer = output['answers'][selected_turn]
    selected_answer = "Yes" if selected_answer == 1 else "No"

    previous_questions = output['questions'][:selected_turn]
    previous_answers = ["Yes" if a == 1 else "No" for a in output['answers'][:selected_turn]]

    actual_relevance_marginal = output['relevance_marginal'][selected_turn]
    actual_informativeness_marginal = output['informativeness_marginal'][selected_turn]
    actual_informativeness_marginal_x_relevance_marginal = output['informativeness_marginal_x_relevance_marginal'][selected_turn]
    actual_informativeness_marginal_x_relevance_marginal_ref = output['informativeness_marginal_x_relevance_marginal_ref'][selected_turn]
                                  
    data_dicts.append(
        {
            "example_id": example_id,
            "procedure": output['procedure'],
            "question": selected_question,
            "answer": selected_answer,
            "previous_questions_answers": [(q, a) for q, a in zip(previous_questions, previous_answers)],
            "nli_metric_scores": {
                "relevance_marginal": actual_relevance_marginal,
                "informativeness_marginal": actual_informativeness_marginal,
                "informativeness_marginal_x_relevance_marginal": actual_informativeness_marginal_x_relevance_marginal,
                "informativeness_marginal_x_relevance_marginal_ref": actual_informativeness_marginal_x_relevance_marginal_ref,
            }
        }
    )

json.dump(data_dicts, open(os.path.join(output_dir, f"informativeness_sample{random_seed}.json"), "w"), indent=4)

In [27]:
# Divide sampled data into a txt file for each annotator
for annotator_idx in range(N_SAMPLES // SAMPLES_PER_ANNOTATOR):
    lines = []

    samples = data_dicts[annotator_idx * SAMPLES_PER_ANNOTATOR:(annotator_idx + 1) * SAMPLES_PER_ANNOTATOR]
    assert len(samples) == SAMPLES_PER_ANNOTATOR
    
    lines.append("Imagine you just had eye surgery, and are unable to see. You're performing a task you're familiar with, but need help to determine whether you successfully completed it.")
    lines.append("You video call a friend (who is unfamiliar with the task) and show them what you're working on. You then ask them some yes/no questions to figure out whether you successfully completed the task.")
    lines.append("")
    lines.append("For each annotation task, you will be given the following information:")
    lines.append(" - A sentence describing the procedure you're trying to perform.")
    lines.append(" - An optional list of previous questions you already asked, and their answers.")
    lines.append(" - The last question you just asked your friend, and its answer.")
    lines.append("")
    lines.append("You must rate how informative the last question and its answer are. By informative, we mean: compared to what you knew from the previous questions and answers, how much more sure would the last question and answer be about whether you succeeded?")
    lines.append("You can also choose to mark 'Instructions Unclear', which means that the instructional text itself is not clear, so you're not sure how to determine whether it's successful. This should only be used in rare cases.")
    lines.append("")
    lines.append("Some tips:")
    lines.append(" - Only judge the informativeness of the last question and answer, not the previous questions and answers (which may or may not be informative).")
    lines.append(" - A question may seem relevant to the task at hand, but you should consider it uninformative if it doesn't provide essential information to judge the success of the task.")
    lines.append(" - If a seemingly informative question is redundant with previous questions, you may consider it less informative.")
    lines.append(" - If the last answer contradicts critical information you had from previous questions and answers, you may consider it more informative.")
    lines.append(' - The instructional text and questions may refer to "someone" or a "person"; always assume this is referring to yourself (the person performing the task).')
    lines.append(' - The questions may refer to a "photo" or "image"; always assume this is referring to the video feed your friend would see through the video call.')
    lines.append("\n")
                 
    # How relevant is this question, i.e., how useful would an answer to this question be in determining whether the procedure has been completed?
    for sample_idx, sample in enumerate(samples):
        lines.append(f"Annotation {sample_idx + 1}")
        lines.append("===============================")
        lines.append(f"Instruction: {sample['procedure']}")
        
        lines.append("Previous questions and answers:")
        if len(sample['previous_questions_answers']) == 0:
            lines.append("None")
        else:
            for q_idx, (q, a) in enumerate(sample['previous_questions_answers']):
                 lines.append(f"Q{q_idx+1}. {q}     (Answer: {a})")
        lines.append("")
        lines.append(f"Last question: {sample['question']}")
        lines.append(f"Last answer: {sample['answer']}")
        lines.append("Your rating (place an 'x' next to your choice):")
        lines.append("1 (very informative): ")
        lines.append("2 (slightly informative): ")
        lines.append("3 (neutral): ")
        lines.append("4 (slightly informative): ")
        lines.append("5 (very informative): ")
        lines.append("Instructions Unclear: ")
        lines.append("\n")
                 
    with open(os.path.join(output_dir, f"informativeness_sample{random_seed}_annotator{annotator_idx+1}.txt"), "w") as f:
        f.write("\n".join(lines))